# Creacion de DataFrame a partir de RDD

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField 
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.types import Row

from pyspark.sql import SQLContext

In [2]:
spark = SparkContext(master='local', appName="TransformacionesYAcciones")
sqlContext = SQLContext(spark)

In [3]:
pwd

'/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark'

In [4]:
!ls ./files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [5]:
path = '/home/rb/Platzi/Escuela de Data Science/22.- Curso Introductorio de Spark/files/'
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv') \
    .map(lambda line: line.split(","))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv') \
    .map(lambda line: line.split(","))

deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [6]:
deportistaOlimpicoRDD.take(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [7]:
# Retiramos encabezado
def eliminarEncabezado(indice, iterador):
    return iter(list(iterador)[1:])

In [8]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.\
        mapPartitionsWithIndex(eliminarEncabezado)

In [9]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [10]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
))

In [11]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [12]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False)
])

In [13]:
deportistaDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [14]:
deportistaDF.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



## Reto crear los dataframes restantes

In [15]:
# Paises Equipos
equiposOlimpicosRDD = spark.textFile(path+'paises.csv') \
    .map(lambda line: line.split(",")) \
    .mapPartitionsWithIndex(eliminarEncabezado) \
    .map(lambda l: (
    int(l[0]),
    l[1],
    l[2]
    ))

equiposSchema = StructType([
    StructField('id', IntegerType(), False),
    StructField('equipo', StringType(), False),
    StructField('sigla', StringType(), False),
])

paisesDF = sqlContext.createDataFrame(equiposOlimpicosRDD, equiposSchema)

paisesDF.show(5)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [16]:
# Deportes
deporteSchema = StructType([
    StructField('deporte_id', IntegerType(), False),
    StructField('deporte', StringType(), False)
])

# Paso 2 Crea DataFrame con la lectura del Schema
deportesDF = sqlContext.read.schema(deporteSchema) \
            .option('header', 'true').csv(path+'deporte.csv')

deportesDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [17]:
# Evento
eventoSchema = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('deporte_id', IntegerType(), False),
    
])

# Paso 2 Crea DataFrame con la lectura del Schema
deportesOlimpicosDF = sqlContext.read.schema(eventoSchema) \
            .option('header', 'true').csv(path+'evento.csv')

deportesOlimpicosDF.show(5)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



In [18]:
# resultados
resultadosSchema = StructType([
    StructField('resultado_id', IntegerType(), False),
    StructField('medalla', StringType(), False),
    StructField('deportista_id', IntegerType(), False),
    StructField('juego_id', IntegerType(), False),
    StructField('evento_id', IntegerType(), False),
    
])

# Paso 2 Crea DataFrame con la lectura del Schema
resultadoDF = sqlContext.read.schema(resultadosSchema) \
            .option('header', 'true').csv(path+'resultados.csv')

resultadoDF.show(5)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [19]:
# juegos
juegoSchema = StructType([
    StructField('juego_id', IntegerType(), False),
    StructField('anio', StringType(), False),
    StructField('temporada', StringType(), False),
    StructField('ciudad', StringType(), False)
])

# Paso 2 Crea DataFrame con la lectura del Schema
juegoDF = sqlContext.read.schema(juegoSchema) \
            .option('header', 'true').csv(path+'juegos.csv')

juegoDF.show(5)

+--------+-----------+---------+------+
|juego_id|       anio|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
|       5|1908 Verano|     1908|Verano|
+--------+-----------+---------+------+
only showing top 5 rows



## Operaciones sobre DF


In [20]:
#ver el Schema
deportesDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [21]:
deportistaDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [22]:
# renombrado de columnas
deportistaOlimpicoDF = deportistaDF \
    .withColumnRenamed("genero", 'sexo') \
    .drop('altura')

In [23]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [24]:
# Usando el rey SELECT
from pyspark.sql.functions import *

deportistaOlimpicoDF = deportistaOlimpicoDF.select("deportista_id", "nombre",
                           col("edad").alias("edadAlJugar"),
                           "equipo_id"
                          )

In [25]:
deportistaOlimpicoDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [26]:
# Usando filer
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(
                        (deportistaOlimpicoDF.edadAlJugar != 0)
                    )

In [27]:
deportistaOlimpicoDF.sort("edadAlJugar").show()

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        70616|          Liu Luyang|         11|      199|
|       118925|Megan Olwen Deven...|         11|      413|
|        52070|        Etsuko Inada|         11|      514|
|        22411|Magdalena Cecilia...|         11|      413|
|        40129|    Luigina Giavotti|         11|      507|
|        47618|Sonja Henie Toppi...|         11|      742|
|        76675|   Marcelle Matthews|         11|      967|
|        37333|Carlos Bienvenido...|         11|      982|
|        51268|      Beatrice Hutiu|         11|      861|
|       126307|        Liana Vicens|         11|      825|
|        48939|             Ho Gang|         12|      738|
|        49142|        Jan Hoffmann|         12|      302|
|        42835|   Werner Grieshofer|         12|       7

## Agrupaciones y operaciones join sobre DF

In [28]:
#visualizamos los Schemas de los DataFrames
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- edadAlJugar: integer (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [29]:
resultadoDF.printSchema()

root
 |-- resultado_id: integer (nullable = true)
 |-- medalla: string (nullable = true)
 |-- deportista_id: integer (nullable = true)
 |-- juego_id: integer (nullable = true)
 |-- evento_id: integer (nullable = true)



In [30]:
juegoDF.printSchema()

root
 |-- juego_id: integer (nullable = true)
 |-- anio: string (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string (nullable = true)



In [31]:
deportesOlimpicosDF.printSchema()

root
 |-- evento_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- deporte_id: integer (nullable = true)



In [32]:
paisesDF.printSchema()

root
 |-- id: integer (nullable = false)
 |-- equipo: string (nullable = false)
 |-- sigla: string (nullable = false)



In [33]:
# JOINS
deportistaOlimpicoDF \
    .join(
    resultadoDF,
    deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id,
    "left"
    ) \
    .join(
    juegoDF,
    juegoDF.juego_id == resultadoDF.juego_id,
    "left"
    ) \
    .join(
    deportesOlimpicosDF,
    deportesOlimpicosDF.evento_id == resultadoDF.evento_id,
    "left"
    ) \
    .select(deportistaOlimpicoDF.nombre, 
            "edadAlJugar",
            "medalla", 
            col("anio").alias("Anio de Juego"),
            deportesOlimpicosDF.nombre.alias("Nombre de disciplina")
           ).show()


+--------------------+-----------+-------+-------------+--------------------+
|              nombre|edadAlJugar|medalla|Anio de Juego|Nombre de disciplina|
+--------------------+-----------+-------+-------------+--------------------+
|           A Dijiang|         24|     NA|  1992 Verano|Basketball Men's ...|
|            A Lamusi|         23|     NA|  2012 Verano|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|         24|     NA|  1920 Verano|Football Men's Fo...|
|Edgar Lindenau Aabye|         34|   Gold|  1900 Verano|Tug-Of-War Men's ...|
|Christine Jacoba ...|         21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1988 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1988 Invierno|Speed Sk

### Reto

Hacer un join con todas las medallas ganadoras unidas con el pais y el equipo al que pertenecen estas medallas.

In [34]:
resultadoDF \
    .join(
    deportistaOlimpicoDF,
    resultadoDF.deportista_id == deportistaOlimpicoDF.deportista_id,
    "left"
    ) \
    .join(
    paisesDF,
    deportistaOlimpicoDF.equipo_id == paisesDF.id,
    "left"    
    ) \
    .select("medalla",
            paisesDF.equipo,
            paisesDF.sigla
    ) \
    .where(resultadoDF.medalla != "NA") .show()

+-------+------+-----+
|medalla|equipo|sigla|
+-------+------+-----+
| Silver| Hi-Hi|  DEN|
| Silver| Hi-Hi|  DEN|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
|   Gold|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Bronze|  Iran|  IRI|
|   Gold|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
|   Gold|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Bronze|  Iran|  IRI|
| Silver|  Iran|  IRI|
| Bronze|  Iran|  IRI|
+-------+------+-----+
only showing top 20 rows



## Solución reto joins
Solucion del profesor

In [35]:
resultadoDF.filter(resultadoDF.medalla != "NA") \
    .join(
        deportistaOlimpicoDF,
        resultadoDF.deportista_id == deportistaOlimpicoDF.deportista_id,
        "left"
    ) \
    .join(
        paisesDF,
        deportistaOlimpicoDF.equipo_id == paisesDF.id,
        "left"    
    ) \
    .select("medalla", paisesDF.equipo, paisesDF.sigla) \
    .sort( col("sigla").desc()) \
    .show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



## Funciones de agrupación


In [36]:
medallistaXAnio = deportistaOlimpicoDF \
    .join(
        resultadoDF, 
        deportistaOlimpicoDF.deportista_id == resultadoDF.deportista_id, 
        "left"
    ) \
    .join(
        juegoDF, 
        juegoDF.juego_id == resultadoDF.juego_id, 
        "left"
    ) \
    .join(
        paisesDF, 
        deportistaOlimpicoDF.equipo_id == paisesDF.id, 
        "left"
    ) \
    .join(
        deportesOlimpicosDF, 
        deportesOlimpicosDF.evento_id == resultadoDF.evento_id, 
        "left"
    ) \
    .join(
        deportesDF, 
        deportesOlimpicosDF.deporte_id == deportesDF.deporte_id, 
        "left"
    ) \
    .select(
        "sigla",
        "anio",
        "medalla",
        deportesOlimpicosDF.nombre.alias("Nombre subdisciplina"),
        deportesDF.deporte.alias("Nombre disciplina"),
        deportistaOlimpicoDF.nombre    
    )

In [37]:
medallistaXAnio.show()

+-----+-------------+-------+--------------------+-----------------+--------------------+
|sigla|         anio|medalla|Nombre subdisciplina|Nombre disciplina|              nombre|
+-----+-------------+-------+--------------------+-----------------+--------------------+
|  BRU|  2016 Verano|     NA|Athletics Women's...|        Athletics|Maizurah Abdul Rahim|
|  BRU|  2000 Verano|     NA|Shooting Men's Skeet|         Shooting|Jefri Kiko Bolkia...|
|  BRU|  1996 Verano|     NA|Shooting Men's Skeet|         Shooting|Jefri Kiko Bolkia...|
|  BRU|  2004 Verano|     NA|Athletics Men's 1...|        Athletics|     Jimmy Anak Ahar|
|  BRU|  2000 Verano|     NA|Athletics Men's 1...|        Athletics|         Haseri Asli|
|  BRU|  2016 Verano|     NA|Athletics Men's 1...|        Athletics|Mohamed Fakhri Is...|
|  BRU|  2012 Verano|     NA|Swimming Men's 20...|         Swimming|Anderson Chee Wei...|
|  BRU|  2012 Verano|     NA|Athletics Women's...|        Athletics|      Maziah Mahusin|
|  BRU|  2

In [38]:
# descartamos deportistas sin medalla
medallistaXAnio2 = medallistaXAnio.filter(resultadoDF.medalla != "NA") \
                    .sort("anio") \
                    .groupBy("sigla", "anio", "Nombre subdisciplina") \
                    .count()

medallistaXAnio2.show()

+-----+-----------+--------------------+-----+
|sigla|       anio|Nombre subdisciplina|count|
+-----+-----------+--------------------+-----+
|  FRA|1896 Verano|Fencing Men's Foi...|    2|
|  FRA|1896 Verano|Cycling Men's 100...|    1|
|  FRA|1896 Verano|Cycling Men's 10,...|    2|
|  FRA|1896 Verano|Cycling Men's Sprint|    2|
|  FRA|1896 Verano|Athletics Men's 1...|    1|
|  FRA|1896 Verano|Cycling Men's 333...|    1|
|  FRA|1896 Verano|Fencing Men's Foi...|    1|
|  FRA|1896 Verano|Athletics Men's T...|    1|
|  SUI|1896 Verano|Gymnastics Men's ...|    1|
|  SUI|1896 Verano|Gymnastics Men's ...|    1|
|  SUI|1896 Verano|Gymnastics Men's ...|    1|
|  GRE|1896 Verano|Gymnastics Men's ...|    1|
|  GRE|1896 Verano|Tennis Men's Doubles|    2|
|  GRE|1896 Verano|Tennis Men's Singles|    2|
|  GRE|1896 Verano|Fencing Men's Sab...|    1|
|  GRE|1896 Verano|Athletics Men's 8...|    1|
|  GRE|1896 Verano|Athletics Men's M...|    1|
|  GRE|1896 Verano|Gymnastics Men's ...|    1|
|  USA|1896 V

In [39]:
medallistaXAnio2.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- Nombre subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [40]:
medallistaXAnio2.groupBy("Sigla", "anio") \
    .agg(sum("count").alias("Total de medallas"), \
        avg("count").alias("Medallas promedio")).show()

# .agg() o agregate es la forma recomendada por la documentacion.

+-----+-----------+-----------------+------------------+
|Sigla|       anio|Total de medallas| Medallas promedio|
+-----+-----------+-----------------+------------------+
|  FRA|1896 Verano|               11|             1.375|
|  SUI|1896 Verano|                3|               1.0|
|  GRE|1896 Verano|                9|1.2857142857142858|
|  USA|1896 Verano|               20|1.6666666666666667|
|  DEN|1896 Verano|                6|               1.0|
|  HUN|1896 Verano|                6|               1.0|
|  GBR|1896 Verano|                8|1.1428571428571428|
|  AUT|1896 Verano|                5|               1.0|
|  AUS|1896 Verano|                3|               1.0|
|  GER|1896 Verano|               30|               2.5|
|  FRA|1900 Verano|              179| 3.314814814814815|
|  GER|1900 Verano|               27|               4.5|
|  SUI|1900 Verano|               21|               3.5|
|  LUX|1900 Verano|                1|               1.0|
|  NED|1900 Verano|            

# SQL

In [41]:
# registro de DataFrames como tablas SQL
resultadoDF.registerTempTable("resultado")
deportistaOlimpicoDF.registerTempTable("deportista")
paisesDF.registerTempTable("paises")

In [42]:
sqlContext.sql("SELECT * FROM deportista").show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [43]:
sqlContext.sql("""
                SELECT medalla, equipo, sigla FROM resultado as r
                JOIN deportista as d
                ON r.deportista_id = d.deportista_id
                JOIN paises as p
                ON p.id = d.equipo_id
                WHERE medalla <> "NA"
                ORDER BY sigla DESC
                """).show(15)

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 15 rows



In [44]:
spark

<SparkContext master=local appName=TransformacionesYAcciones>

# UDF

In [45]:
!ls ./files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [46]:
!head -n 5 ./files/deportistaError.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,,,273
4,Edgar Lindenau Aabye,1,34,,,278


En mi caso muy particular tuve que recurrir a cargar el DataFrame con sqlContext, ya que la libreria py4j generaba un error al introducir los tipos de dato row del rdd altura='', la forma presentada con el aporte de Jose Daniel Gallegos permite hacer conversion de datos de forma automatica

In [47]:
DeportistaError_schema = StructType([
    StructField("deportista_id", StringType(), False),
    StructField("nombre", StringType(), False),
    StructField("genero", StringType(), False),
    StructField("edad", StringType(), False),
    StructField("altura", StringType(), False), 
    StructField("peso", StringType(), False),
    StructField("equipo_id", StringType(), False)
])

deportistaErrorDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(DeportistaError_schema).\
            load(path+"deportistaError.csv")

In [48]:
deportistaErrorDF.show()

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|  80|      199|
|            2|            A Lamusi|     1|  23|   170|  60|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|  null|null|      273|
|            4|Edgar Lindenau Aabye|     1|  34|  null|null|      278|
|            5|Christine Jacoba ...|     2|  21|   185|  82|      705|
|            6|     Per Knut Aaland|     1|  31|   188|  75|     1096|
|            7|        John Aalberg|     1|  31|   183|  72|     1096|
|            8|"Cornelia ""Cor""...|     2|  18|   168|null|      705|
|            9|    Antti Sami Aalto|     1|  26|   186|  96|      350|
|           10|"Einar Ferdinand ...|     1|  26|  null|null|      350|
|           11|  Jorma Ilmari Aalto|     1|  22|   182|76.5|      350|
|     

Si analizamos la columna altura nosotros queremos que sean enteros y en ausencia del valor deberia aparecer un null, aqui entran en el juego las **UDF**, las cuales trabajaran de forma nativa

In [49]:
from pyspark.sql.functions import udf

In [52]:
def conversionEnteros(valor):
    try:
        return int(valor) if len(valor) > 0 else 0
    except TypeError as e:
        return 0
    
conversionEnteros_udf = udf(lambda z: conversionEnteros(z), IntegerType())
sqlContext.udf.register("conversionEnteros_udf", conversionEnteros_udf)

deportistaErrorDF.select(conversionEnteros_udf("altura") \
                         .alias("alturaUDF")).show()

+---------+
|alturaUDF|
+---------+
|      180|
|      170|
|        0|
|        0|
|      185|
|      188|
|      183|
|      168|
|      186|
|        0|
|      182|
|      172|
|      159|
|      171|
|        0|
|      184|
|      175|
|      189|
|        0|
|      176|
+---------+
only showing top 20 rows



# DF y replicacion

In [58]:
from pyspark.storagelevel import StorageLevel

In [59]:
medallistaXAnio.is_cached

False

Cada vez que necesitamos el recurso Spark necesita "incovarlo" de nuevo, pero podemos indicarle a spark que lo guarde en cache.

In [60]:
# Creamos la persisttencia
medallistaXAnio.rdd.cache()

MapPartitionsRDD[264] at javaToPython at NativeMethodAccessorImpl.java:0

In [61]:
# Saber el tipo de persistencia y nivel de replicacion
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(False, True, False, False, 1)

In [62]:
# eliminando la persistencia
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[264] at javaToPython at NativeMethodAccessorImpl.java:0

In [63]:
# Usando persistencia MEMORY AND DISK con replicacion 2
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_2)

MapPartitionsRDD[264] at javaToPython at NativeMethodAccessorImpl.java:0

In [64]:
# Saber el tipo de persistencia y nivel de replicacion
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 2)

In [65]:
# eliminando la persistencia
medallistaXAnio.rdd.unpersist()

MapPartitionsRDD[264] at javaToPython at NativeMethodAccessorImpl.java:0

In [66]:
# Definiendo un StorageLevel custom
StorageLevel.MEMORY_AND_DISK_3 = StorageLevel(True, True, False, False, 3)

In [67]:
# Persistimos el DataFrame
medallistaXAnio.rdd.persist(StorageLevel.MEMORY_AND_DISK_3)

MapPartitionsRDD[264] at javaToPython at NativeMethodAccessorImpl.java:0

In [68]:
# Saber el tipo de persistencia y nivel de replicacion
medallistaXAnio.rdd.getStorageLevel()

StorageLevel(True, True, False, False, 3)